## 1. Creating Hashed Passwords for Authentication:

* Everytime when a new user is added to the app, we need to get the user_id (usually email) and their password from them.
* Then we need to create a `Hash` of the password and add it to the `authenticator.yml` file in order for the user to be able to log-in to the app.

In [4]:
from streamlit_authenticator.utilities.hasher import Hasher

hashed_passwords = Hasher(['abc123']).generate()
print(hashed_passwords) 

['$2b$12$tdKt5p7nfhQgcDG30FldqeTZLUio5P.vAqKXSqiUHxURaei7Npkq2']


In [5]:
hashed_passwords = Hasher(['blackpearl']).generate()
print(hashed_passwords) 

['$2b$12$N9w9IGLBfLv1tne8lzalUOm2hH4ytnvbTgQ.vs0jV6EbKt9x2QPr6']


## 2. Testing the connection with Snowflake Data warehouse:

In [32]:
import snowflake.connector
from dotenv import load_dotenv
import os 

load_dotenv()

True

In [34]:
con = snowflake.connector.connect(
    user='hariharan',
    password= os.getenv('SNOWSQL_PWD'),
    account=os.getenv('SNOWSQL_ACCOUNT'),
    warehouse = os.getenv('SNOWSQL_WAREHOUSE'),
)

In [35]:
# Show all the databses present in the database
cur = con.cursor()
cur.execute("SHOW DATABASES")
result = cur.fetchall()
for row in result:
    schema_name = row[1]
    print(schema_name)

FOOD_DELIVERY
SNOWFLAKE
SNOWFLAKE_SAMPLE_DATA


In [36]:
# Show all the schemas present in the database
cur = con.cursor()
cur.execute("SHOW SCHEMAS IN FOOD_DELIVERY")
result = cur.fetchall()
for row in result:
    schema_name = row[1]
    print(schema_name)

CORE
INFORMATION_SCHEMA
PUBLIC


In [37]:
# Show all the tables present in the schema CORE
cur.execute("SHOW TABLES IN SCHEMA FOOD_DELIVERY.CORE")
result = cur.fetchall()
for row in result:
    table_name = row[1]
    print(table_name)

MENU_ITEMS
ORDERS
ORDER_DETAILS
PAYMENTS
RESTAURANTS
REVIEWS
USERS


In [38]:
# creating cursor object
cur = con.cursor()

# Execute a statement that will generate a result set.
sql = "SELECT * FROM FOOD_DELIVERY.CORE.ORDERS"
cur.execute(sql)

# Fetch the result set from the cursor and deliver it as the pandas DataFrame.
df = cur.fetch_pandas_all()

print(df.shape)
df.head()

(73748, 7)


,ORDER_ID,USER_ID,ORDER_TIME,DELIVERY_ADDRESS,ORDER_STATUS,RESTAURANT_ID,TOTAL_AMOUNT
0,716d868b-2b5a-4bee-bc2c-ee262c81588f,d36e1a86-6e33-434a-b9c7-3f5c30753402,2024-06-01 11:53:19,"9198 Gabriela Green\nEast Marcton, DC 16873",Delivered,R18329211,152.85
1,0f41a3a7-954a-4e24-ad84-dba22e8dd154,1ae81af6-6b78-4695-8996-442e9e40ad3c,2024-06-01 08:11:05,"366 Byrd Hills\nNew Robert, WI 24455",Delivered,R22670500,191.50
2,fde175ad-d2ea-4901-a5fd-5c28f44e7382,2c1492fd-c993-4d00-9086-dc105c821bae,2024-06-01 21:46:46,"71211 Gregory Track\nGreenestad, OH 72514",Delivered,R66375744,35.88
3,99e370cd-f55e-4793-b0e6-0c41d8fd9287,523d1f1a-9edd-4811-9472-f65a52d51f15,2024-06-01 09:04:39,"070 Steven Heights\nRoachville, PW 12962",Delivered,R42644875,149.57
4,a88b8bf1-9fdf-4c8e-87ab-ac3a390d05e0,a6c31749-eb2a-40f3-aa65-49ead5cfdc3a,2024-06-01 21:05:23,"9382 Alyssa Branch\nShellyfurt, VI 11466",Delivered,R23024716,129.32


## 3. Testing the connection with Databricks Data warehouse:

In [53]:
from dotenv import load_dotenv
import os 
import pandas as pd 
from databricks import sql 

load_dotenv()

True

In [54]:
# Creating a connection to databricks
connection = sql.connect(server_hostname=os.getenv("DATABRICKS_SERVER_HOST_NAME"),
                         http_path=os.getenv("DATABRICKS_HTTP_PATH"),
                         access_token=os.getenv("DATABRICKS_ACCESS_TOKEN")
                         )


In [56]:
df = pd.read_sql("SELECT * FROM hive_metastore.online_food_business.menu_items", connection)
print(df.shape)
df.head()

/var/folders/3g/7x619qt169344pmm32qd8wgc0000gq/T/ipykernel_8801/705794241.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM hive_metastore.online_food_business.menu_items", connection)


(3000, 4)


,menu_item_id,restaurant_id,item_name,price
0,R32379007_1,R32379007,Tacos,10.20
1,R32379007_2,R32379007,Burritos,13.93
2,R32379007_3,R32379007,Enchiladas,14.15
3,R32379007_4,R32379007,Quesadillas,10.47
4,R32379007_5,R32379007,Nachos,10.80


## 4. Retrieving the Table Schema, Categorical Column details and Sample rows for Prompt Context:

In [3]:
from dotenv import load_dotenv
import os 
import pandas as pd 
from databricks import sql 

load_dotenv()

True

In [5]:
# Selected tables list 
catalog = "hive_metastore"
schema = "online_food_business"
tables_list = ["menu_items","orders","users"]

In [52]:
table_schema = ""

# Iterating through each selected tables and get the list of columns for each table.
for table in tables_list:

    conn = sql.connect(server_hostname = os.getenv("DATABRICKS_SERVER_HOSTNAME"),
                    http_path       = os.getenv("DATABRICKS_HTTP_PATH"),
                    access_token    = os.getenv("DATABRICKS_ACCESS_TOKEN"))        

    # Getting the Schema for the table
    query = f"SHOW CREATE TABLE `{catalog}`.{schema}.{table}"
    df = pd.read_sql(sql=query,con=conn)
    stmt = df['createtab_stmt'][0]
    stmt = stmt.split("USING")[0]

    # Filtering the String columns from the table to identify Categorical columns
    query = f"DESCRIBE TABLE `{catalog}`.{schema}.{table}"
    df = pd.read_sql(sql=query,con=conn)
    string_cols = df[df['data_type']=='string']['col_name'].values.tolist()

    sql_distinct = ""
    for col in string_cols:
        # Getting the distinct values for each column as rows
        if col == string_cols[-1]:
            sql_distinct += f"SELECT '{col}' AS column_name, COUNT(DISTINCT {col}) AS cnt, ARRAY_AGG(DISTINCT {col}) AS values FROM `{catalog}`.{schema}.{table}"
        else:
            sql_distinct += f"SELECT '{col}' AS column_name, COUNT(DISTINCT {col}) AS cnt, ARRAY_AGG(DISTINCT {col}) AS values FROM `{catalog}`.{schema}.{table} UNION ALL "

    # print(sql_distinct)
    df_categories = pd.read_sql(sql=sql_distinct,con=conn)
    df_categories = df_categories[df_categories['cnt'] <= 20]
    df_categories = df_categories.drop(columns='cnt')

    if df_categories.empty:
        df_categories_string = "No Categorical Fields"
    else:
        df_categories_string = df_categories.to_string(index=False)


    # Getting the sample rows from the table
    query = f"SELECT * FROM `{catalog}`.{schema}.{table} LIMIT 3"
    df = pd.read_sql(sql=query,con=conn)
    sample_rows = df.to_string(index=False)

    
    # df_categories = df_string.groupby('col_name').filter(lambda x: x['col_name'].count() <= 20)
    # print(df_string)
    # print(df_categories)
    if table_schema == "":
        table_schema = stmt + "\n" + sample_rows + "\n\nCategorical Fields:\n" + df_categories_string + "\n"
    else:
        table_schema = table_schema + "\n" + stmt + "\n" + sample_rows + "\n\nCategorical Fields:\n" + df_categories_string + "\n"

/var/folders/3g/7x619qt169344pmm32qd8wgc0000gq/T/ipykernel_19961/325098163.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=query,con=conn)
/var/folders/3g/7x619qt169344pmm32qd8wgc0000gq/T/ipykernel_19961/325098163.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=query,con=conn)
/var/folders/3g/7x619qt169344pmm32qd8wgc0000gq/T/ipykernel_19961/325098163.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_categories = pd.read_sql(sql=sql_distinct,con=conn)
/var/folders/3g/

In [53]:
print(table_schema)

CREATE TABLE hive_metastore.online_food_business.menu_items (
  menu_item_id STRING,
  restaurant_id STRING,
  item_name STRING,
  price DOUBLE)

menu_item_id restaurant_id  item_name  price
 R32379007_1     R32379007      Tacos  10.20
 R32379007_2     R32379007   Burritos  13.93
 R32379007_3     R32379007 Enchiladas  14.15

Categorical Fields:
No Categorical Fields

CREATE TABLE hive_metastore.online_food_business.orders (
  order_id STRING,
  user_id STRING,
  order_time TIMESTAMP,
  delivery_address STRING,
  order_status STRING,
  restaurant_id STRING,
  total_amount DOUBLE)

                            order_id                              user_id                order_time                            delivery_address order_status restaurant_id  total_amount
716d868b-2b5a-4bee-bc2c-ee262c81588f d36e1a86-6e33-434a-b9c7-3f5c30753402 2024-06-01 11:53:19+00:00 9198 Gabriela Green\nEast Marcton, DC 16873    Delivered     R18329211        152.85
0f41a3a7-954a-4e24-ad84-dba22e8dd154 1ae81a

In [54]:
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.chains.llm import LLMChain
from langchain_openai import ChatOpenAI

def create_sql(question,table_schema):

    ### Defining the prompt template
    template_string = """ 
    You are a expert data engineer working with a Databricks environment.\
    Your task is to generate a working SQL query in Databricks SQL dialect. \
    During join if column name are same please use alias ex llm.customer_id \
    in select statement. It is also important to respect the type of columns: \
    if a column is string, the value should be enclosed in quotes. \
    If you are writing CTEs then include all the required columns. \
    While concatenating a non string column, make sure cast the column to string. \
    For date columns comparing to string , please cast the string input.\
    For string columns, check if it is a categorical column and only use the appropriate values provided in the schema.\

    SCHEMA:
    ## {table_schema} ##

    QUESTION:
    ##
    {question}
    ##


    IMPORTANT: MAKE SURE THE OUTPUT IS JUST THE SQL CODE AND NOTHING ELSE. Ensure the appropriate CATALOG is used in the query and SCHEMA is specified when reading the tables.
    ##

    OUTPUT:
    """
    prompt_template = PromptTemplate.from_template(template_string)

    ### Defining the LLM chain
    llm_chain = LLMChain(
        llm=ChatOpenAI(model="gpt-4o-mini",temperature=0),
        prompt=prompt_template
    )

    response =  llm_chain.invoke({"question":question,"table_schema":table_schema})
    output = response['text']

    return output

In [59]:
result = create_sql("What is the total orders by different gender and how many unique users for each",table_schema)

In [60]:
print(result)

```sql
WITH order_summary AS (
    SELECT 
        u.gender AS user_gender,
        COUNT(o.order_id) AS total_orders,
        COUNT(DISTINCT o.user_id) AS unique_users
    FROM 
        hive_metastore.online_food_business.orders o
    JOIN 
        hive_metastore.online_food_business.users u 
    ON 
        o.user_id = u.user_id
    GROUP BY 
        u.gender
)

SELECT 
    user_gender,
    total_orders,
    unique_users
FROM 
    order_summary
```


In [62]:
# Function to create enriched database schema details for the Prompt
def get_enriched_database_schema(catalog,schema,tables_list):
    table_schema = ""

    # Iterating through each selected tables and get the list of columns for each table.
    for table in tables_list:

        conn = sql.connect(server_hostname = os.getenv("DATABRICKS_SERVER_HOSTNAME"),
                        http_path       = os.getenv("DATABRICKS_HTTP_PATH"),
                        access_token    = os.getenv("DATABRICKS_ACCESS_TOKEN"))        

        # Getting the Schema for the table
        query = f"SHOW CREATE TABLE `{catalog}`.{schema}.{table}"
        df = pd.read_sql(sql=query,con=conn)
        stmt = df['createtab_stmt'][0]
        stmt = stmt.split("USING")[0]

        # Filtering the String columns from the table to identify Categorical columns
        query = f"DESCRIBE TABLE `{catalog}`.{schema}.{table}"
        df = pd.read_sql(sql=query,con=conn)
        string_cols = df[df['data_type']=='string']['col_name'].values.tolist()

        sql_distinct = ""
        for col in string_cols:
            # Getting the distinct values for each column as rows
            if col == string_cols[-1]:
                sql_distinct += f"SELECT '{col}' AS column_name, COUNT(DISTINCT {col}) AS cnt, ARRAY_AGG(DISTINCT {col}) AS values FROM `{catalog}`.{schema}.{table}"
            else:
                sql_distinct += f"SELECT '{col}' AS column_name, COUNT(DISTINCT {col}) AS cnt, ARRAY_AGG(DISTINCT {col}) AS values FROM `{catalog}`.{schema}.{table} UNION ALL "

        # print(sql_distinct)
        df_categories = pd.read_sql(sql=sql_distinct,con=conn)
        df_categories = df_categories[df_categories['cnt'] <= 20]
        df_categories = df_categories.drop(columns='cnt')

        if df_categories.empty:
            df_categories_string = "No Categorical Fields"
        else:
            df_categories_string = df_categories.to_string(index=False)


        # Getting the sample rows from the table
        query = f"SELECT * FROM `{catalog}`.{schema}.{table} LIMIT 3"
        df = pd.read_sql(sql=query,con=conn)
        sample_rows = df.to_string(index=False)

        if table_schema == "":
            table_schema = stmt + "\n" + sample_rows + "\n\nCategorical Fields:\n" + df_categories_string + "\n"
        else:
            table_schema = table_schema + "\n" + stmt + "\n" + sample_rows + "\n\nCategorical Fields:\n" + df_categories_string + "\n"
    
    return table_schema